<a href="https://colab.research.google.com/github/lmmlima/ENV716_EnergyModeling_F2021/blob/main/Lab6/Lab6_MoreonLPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 6 - More on LPs in Python**

Learning outcomes for Lab 6:
* Learn how to implement a simple LP using numpy arrays;
* Learn how to implement a simple LP using object **Sets()** and **Param()**;
* Learn how to use dictionaries and UDFs in Pyomo to facilitate model implementation.


## Initializing 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

Installing Pyomo and glpk solver.

In [3]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

     |████████████████████████████████| 9.1 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

Importing pyomo and solver.

In [4]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('glpk')

## Implementing Model (Approach #3 - preferred method)




We will implement the chemical solutions model again using model objects (Sets() and Param()), using dictionaries to specify parameters, indexing decision variables and constraints by sets, using user defined function to define constraints and objective function in a more general form.

Start by writing the LP formulation indexed by sets and in standard form. 

Sets: \\
$m∈ M:$ set of machines A and B \\
$p∈P:$ set of chemical solution types I and II

Parameters:  \\
$a_{m,p}:$ number of hours on machine m needed to produce chemical solution type $p$ \\
$H_m:$  number of hours available on machine $m$ \\
$C_p:$ profit gained from producing type $p$ \\

Decision Variables: \\
$x_p:$ number of units of type p to produce \\

Problem Formulation: \\  
$ max_x	\sum_{p\in P}c_p*x_p$ \\
$ s.t.	\sum_{p\in P}a_{m,p}*x_p ≤ H_m  \quad ∀ m \in M $ \\
$ \quad \quad \quad x_p ≥ 0 \quad \quad \quad \quad   ∀p\in P $ \\

Now let’s start by adding sets and parameters to the model. Because we are using objects Sets() and Param(), the assigned names should start with “model.”.

In [5]:
#using sets and parameter
model=ConcreteModel()

#Sets
model.M=Set(initialize=['MA','MB'])  #set of machines
model.P=Set(initialize=['TypeI','TypeII']) #set of solution types

#Parameters
model.c=Param(model.P,initialize={'TypeI':800,'TypeII':600})
model.H=Param(model.M,initialize={'MA':60,'MB':48})
model.a=Param(model.M,model.P,initialize={
    ('MA','TypeI'):4,
    ('MA','TypeII'):2,
    ('MB','TypeI'):2,
    ('MB','TypeII'):4})

Next define the decision variables, objective function and constraint also indexed by set. For generalization purpose note that we use user defined function to enter the expressions. Most of the formulation you will find on the internet will use this syntax. Don’t forget object “model” will always be an argument to your function. And if you need to add a constraint for all elements within a specific set, those elements should also be an argument to your function. Note that now instead if using the “exp=” we are using “rule=”.

In [6]:
#add dec variables
model.X=Var(model.P,domain=NonNegativeReals)

#add obj func
def obj_profit(model):
    return sum(model.c[p]*model.X[p] for p in model.P)
model.profit=Objective(sense=maximize,rule=obj_profit)

#add const
def mach_hours(model,m):
    return sum(model.a[m,p]*model.X[p] for p in model.P) <= model.H[m]
model.mach=Constraint(model.M,rule=mach_hours)


Then all you need to do is solve the model and print the results.

In [7]:
#Solve model
results=opt.solve(model)

#Print results
print("Profit =",model.profit())
print("Decision Variables")
for p in model.P:
    print(model.X[p],model.X[p].value)

Profit = 13200.0
Decision Variables
X[TypeI] 12.0
X[TypeII] 6.0


## Exercise 1

Imagine now we have a third type o chemical solution. How would you change the model above to incorporate a third type that has a profit of 700 and takes 2 hours on machine A and 2 hours on machine B.